In [ ]:
from tiff_utils import *

import matplotlib.pyplot as plt
from rasterio.plot import show
from glob import glob
import pandas as pd
import numpy as np
import rasterio
import os

In [ ]:
src_root_data_dir = '/Users/beppe/Projects/AgriTech/Sentinel-2/2021/'  # Sentinel2 data folder
tiff_root_data_dir = '/Users/beppe/Projects/AgriTech/TiffData/2021/'  # output data folder

bands = ["B02", "B03", "B04", "B05", "B08", "B8A", "B11", "B12"]    # definire le bande che si vogliono utilizzare
resolutions = ["R10m", "R10m", "R10m", "R20m", "R10m", "R20m", "R20m", "R20m"] # e la relativa risoluzione
bands_and_resolutions = list(zip(bands, resolutions))

target_dim = (10980, 10980)

In [ ]:
my_points = [
    (45.10, 10.98),
    (45.10, 11.14),
    (44.99, 11.14),
    (44.99, 10.98)
]

In [ ]:
my_poly = get_test_polygon(my_points)

In [ ]:
src_data_dirs = []
# per ogni giorno a disposizione individua il path in cui recuperare i dati
#for x in os.listdir(src_root_data_dir):
for x in glob(os.path.join(src_root_data_dir, '*')):
    # date = x.split('/')[-1].split("_")[2].split("T")[0]
    name = x.split('/')[-1].split(".")[0]
    src_data_dir = glob(os.path.join(x, "GRANULE/*/IMG_DATA"))[0]
    src_data_dirs.append((name, src_data_dir))

src_data_dirs = sorted(src_data_dirs, key=lambda x: x[0])

In [ ]:
for date, src_data_dir in src_data_dirs:
    tiff_file = os.path.join(tiff_root_data_dir, date + ".tiff")
    
    if os.path.exists(tiff_file):
        continue
    
    tiff_f = None
    
    for i, (band, resolution) in enumerate(bands_and_resolutions, start=1):
        band_file = glob(os.path.join(src_data_dir, resolution, "*_" + band + "_*.jp2"))[0]
        
        band_f = rasterio.open(band_file, driver="JP2OpenJPEG")
        band_data = band_f.read(1)

        if band_data.shape[0] < target_dim[0] and band_data.shape[1] < target_dim[1]:
            print("Extrapolating", band_data.shape, "to", target_dim)
            band_data = extrapolate(band_data, target_dim).astype(band_f.dtypes[0])
            
        if tiff_f is None:  
            profile = band_f.profile
            profile.update(driver="Gtiff", count=len(bands_and_resolutions))
            tiff_f = MemoryFile().open(**profile)
            
        print("Writing band {} for date {}".format(band, date))
        tiff_f.write(band_data, i)
        
        band_f.close()
        
        
    tiff_f_cropped = crop_memory_tiff_file(tiff_f, [my_poly], crop=True)
    
    tiff_f.close()
    tiff_f = None

    with rasterio.open(tiff_file, "w", **tiff_f_cropped.profile) as f:
        f.write(tiff_f_cropped.read())
        
    tiff_f_cropped.close()

In [ ]:
aux = glob(tiff_root_data_dir + '/*')
aux

In [ ]:
image1 = rgb_from_tiff(aux[1])
image2 = rgb_from_tiff(aux[4])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 8))
show(image1, ax=ax[0], cmap='coolwarm_r')
show(image2, ax=ax[1], cmap='coolwarm_r')
plt.show()